# Coursera Applied Data Science Capstone Project: Battle of the Neighborhoods

## Introduction

*Clearly define a problem or an idea of your choice, where you would need to leverage the Foursquare location data to solve or execute. Remember that data science problems always target an audience and are meant to help a group of stakeholders solve a problem, so make sure that you explicitly describe your audience and why they would care about your problem.*

**Currently, the COVID-19 virus has spread across the globe, resulting in the latest pandemic. Since March 2020, New York City has seen rapid increase in confirmed cases, leading to a "shelter in place" order issued by the state officials. This means that healthy or sick, all New Yorkers must stay home unless they are essential workers or need urgent health care. All non-essential businesses that are normally open to the public must remain closed. Which neighborhoods are more affected by the latest orders? It would be helpful to understand which businesses are closed / remain open, in order to better redirect resources to support each of the neighborhoods.**

## Dataset

*Describe the data that you will be using to solve the problem or execute your idea. Remember that you will need to use the Foursquare location data to solve the problem or execute your idea. You can absolutely use other datasets in combination with the Foursquare location data. So make sure that you provide adequate explanation and discussion, with examples, of the data that you will be using, even if it is only Foursquare location data.*

**In this exercise, Foursquare API data of New York City will be used. The explore function will be used to understand the venue types and counts in each neighborhood. Then based on the venue categories, we can assess the imapct of a "shelter in place" order on each neighborhood, mainly whether there are essential businesses (pharmacies and grocery stores, for example) in the area.**

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [102]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


### Dataset 1: NYC location data
We first need to access to a dataset that contains New York City boroughs and the neighborhoods that exist in each of the boroughs, including their latitude and longtitude coordinates in order to segment them.

In [103]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [104]:
# load the data
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [105]:
# preview data set
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [106]:
# define new variable for new data
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

The next task is to transform this data of nested Python dictionaries into a pandas dataframe.

In [107]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

# make sure data is loaded as intended
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [108]:
# loop through data and fill dataframe one row at a time
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [109]:
# preview the dataframe
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [110]:
# confirm the counts of boroughs and neightborhoods
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


Use geopy library to get the latitude and longitude values of New York City.

In [111]:
# In order to define an instance of the geocoder, we need to define a user_agent.
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


### Dataset 2: Venue data from Foursquare
We are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [112]:
# first connect to Foursquare
CLIENT_ID = 'XYBBZ53ERCZVWM31JLR23FENUE52G0DLBDHQZ3NCV2HZAVNO' # your Foursquare ID
CLIENT_SECRET = 'FI5Z1KGO3PHTXFB1TQZGJXGGLXZZARJIEU23MXN0CCDU0ZUS' # your Foursquare Secret
VERSION = '20200323' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: XYBBZ53ERCZVWM31JLR23FENUE52G0DLBDHQZ3NCV2HZAVNO
CLIENT_SECRET:FI5Z1KGO3PHTXFB1TQZGJXGGLXZZARJIEU23MXN0CCDU0ZUS


We first explore a single neighborhood in NYC to verify that the two datasets that we have identified could be used together. For the sake of simplicity we focus on the central area in NYC, Manhattan.

In [113]:
# filter for manhattan only and reset index
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [114]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [115]:
# get the first neighborhood in manhattan
manhattan_data.loc[0, 'Neighborhood']

'Marble Hill'

In [116]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

In [117]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT) #create URL
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=XYBBZ53ERCZVWM31JLR23FENUE52G0DLBDHQZ3NCV2HZAVNO&client_secret=FI5Z1KGO3PHTXFB1TQZGJXGGLXZZARJIEU23MXN0CCDU0ZUS&v=20200323&ll=40.87655077879964,-73.91065965862981&radius=500&limit=100'

In [118]:
# send ET request to Foursquare
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e7c51a8f7706a0023eb5adf'},
 'response': {'headerLocation': 'Marble Hill',
  'headerFullLocation': 'Marble Hill, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 24,
  'suggestedBounds': {'ne': {'lat': 40.88105078329964,
    'lng': -73.90471933917806},
   'sw': {'lat': 40.87205077429964, 'lng': -73.91659997808156}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b4429abf964a52037f225e3',
       'name': "Arturo's",
       'location': {'address': '5198 Broadway',
        'crossStreet': 'at 225th St.',
        'lat': 40.87441177110231,
        'lng': -73.91027100981574,
        'labeledLatLngs': [{'label': 'display',
          'lat': 40.87441177110231,
          'lng': -73.91027100981574}],
        'distance'

In [119]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [120]:
# clean the json file and transform into a pandas dataframe
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Dunkin',Donut Shop,40.877136,-73.906666
4,Starbucks,Coffee Shop,40.877531,-73.905582


In [121]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

24 venues were returned by Foursquare.


We can then extend this analysis to all neighborhoods in Manhattan.

In [122]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [123]:
# run the above code for venues in all neighborhoods in Manhattan
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [124]:
# check the size of the dataframe
print(manhattan_venues.shape)
manhattan_venues.head()

(3312, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
4,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop


In [125]:
# check venue counts returned for each of the neighborhoods
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,94,94,94,94,94,94
Carnegie Hill,100,100,100,100,100,100
Central Harlem,43,43,43,43,43,43
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,44,44,44,44,44,44
East Village,100,100,100,100,100,100


In [126]:
# check unique venue categories in neighborhoods of Manhattan
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 341 uniques categories.


We now analyze venue categories by neigborhood.

In [127]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Gym,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pop-Up Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scan

In [128]:
# select categories that we care about, namely categories that are essential - Convenience Store, Grocery Store, Organic Grocery, Pharmacy, Supermarket
manhattan_essentials = manhattan_onehot[['Neighborhood','Convenience Store','Grocery Store','Organic Grocery','Pharmacy','Supermarket']]
manhattan_essentials.head()

,Neighborhood,Convenience Store,Grocery Store,Organic Grocery,Pharmacy,Supermarket
0,Marble Hill,0,0,0,0,0
1,Marble Hill,0,0,0,0,0
2,Marble Hill,0,0,0,0,0
3,Marble Hill,0,0,0,0,0
4,Marble Hill,0,0,0,0,0


In [129]:
print('Size of the one hot encoded dataframe is',manhattan_onehot.shape)

Size of the one hot encoded dataframe is (3312, 342)


In [130]:
print('Size of the essentials-only dataframe is',manhattan_essentials.shape)

Size of the essentials-only dataframe is (3312, 6)


In [131]:
# Check essentials venue counts for each neighborhood
manhattan_essentials.groupby('Neighborhood').count()

,Convenience Store,Grocery Store,Organic Grocery,Pharmacy,Supermarket
Neighborhood,,,,,
Battery Park City,94,94,94,94,94
Carnegie Hill,100,100,100,100,100
Central Harlem,43,43,43,43,43
Chelsea,100,100,100,100,100
Chinatown,100,100,100,100,100
Civic Center,100,100,100,100,100
Clinton,100,100,100,100,100
East Harlem,44,44,44,44,44
East Village,100,100,100,100,100


In [132]:
manhattan_grouped = manhattan_essentials.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Convenience Store,Grocery Store,Organic Grocery,Pharmacy,Supermarket
0,Battery Park City,0.000000,0.021277,0.00,0.000000,0.000000
1,Carnegie Hill,0.000000,0.030000,0.00,0.000000,0.020000
2,Central Harlem,0.000000,0.000000,0.00,0.000000,0.000000
3,Chelsea,0.000000,0.010000,0.00,0.000000,0.000000
4,Chinatown,0.000000,0.000000,0.01,0.000000,0.010000
5,Civic Center,0.000000,0.000000,0.00,0.000000,0.000000
6,Clinton,0.000000,0.010000,0.01,0.000000,0.010000
7,East Harlem,0.022727,0.022727,0.00,0.022727,0.000000
8,East Village,0.000000,0.000000,0.01,0.000000,0.000000
9,Financial District,0.010000,0.010000,0.00,0.000000,0.000000


In [133]:
# print new neighborhood along with top 5 most common venues

num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
               venue  freq
0      Grocery Store  0.02
1  Convenience Store  0.00
2    Organic Grocery  0.00
3           Pharmacy  0.00
4        Supermarket  0.00


----Carnegie Hill----
               venue  freq
0      Grocery Store  0.03
1        Supermarket  0.02
2  Convenience Store  0.00
3    Organic Grocery  0.00
4           Pharmacy  0.00


----Central Harlem----
               venue  freq
0  Convenience Store   0.0
1      Grocery Store   0.0
2    Organic Grocery   0.0
3           Pharmacy   0.0
4        Supermarket   0.0


----Chelsea----
               venue  freq
0      Grocery Store  0.01
1  Convenience Store  0.00
2    Organic Grocery  0.00
3           Pharmacy  0.00
4        Supermarket  0.00


----Chinatown----
               venue  freq
0    Organic Grocery  0.01
1        Supermarket  0.01
2  Convenience Store  0.00
3      Grocery Store  0.00
4           Pharmacy  0.00


----Civic Center----
               venue  freq
0  Convenience Store   0.0


In [134]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Battery Park City,Grocery Store,Supermarket,Pharmacy,Organic Grocery,Convenience Store
1,Carnegie Hill,Grocery Store,Supermarket,Pharmacy,Organic Grocery,Convenience Store
2,Central Harlem,Supermarket,Pharmacy,Organic Grocery,Grocery Store,Convenience Store
3,Chelsea,Grocery Store,Supermarket,Pharmacy,Organic Grocery,Convenience Store
4,Chinatown,Supermarket,Organic Grocery,Pharmacy,Grocery Store,Convenience Store


In [135]:
# print new neighborhood along with essential venues

manhattan_ess_grouped = manhattan_essentials.groupby('Neighborhood').mean().reset_index()
manhattan_ess_grouped

,Neighborhood,Convenience Store,Grocery Store,Organic Grocery,Pharmacy,Supermarket
0,Battery Park City,0.000000,0.021277,0.00,0.000000,0.000000
1,Carnegie Hill,0.000000,0.030000,0.00,0.000000,0.020000
2,Central Harlem,0.000000,0.000000,0.00,0.000000,0.000000
3,Chelsea,0.000000,0.010000,0.00,0.000000,0.000000
4,Chinatown,0.000000,0.000000,0.01,0.000000,0.010000
5,Civic Center,0.000000,0.000000,0.00,0.000000,0.000000
6,Clinton,0.000000,0.010000,0.01,0.000000,0.010000
7,East Harlem,0.022727,0.022727,0.00,0.022727,0.000000
8,East Village,0.000000,0.000000,0.01,0.000000,0.000000
9,Financial District,0.010000,0.010000,0.00,0.000000,0.000000


In [136]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 3, 0, 0, 0, 0, 0, 1, 0, 0], dtype=int32)

In [137]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge data and add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# check the last columns!
manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,2,Pharmacy,Supermarket,Organic Grocery,Grocery Store,Convenience Store
1,Manhattan,Chinatown,40.715618,-73.994279,0,Supermarket,Organic Grocery,Pharmacy,Grocery Store,Convenience Store
2,Manhattan,Washington Heights,40.851903,-73.936900,3,Grocery Store,Supermarket,Pharmacy,Organic Grocery,Convenience Store
3,Manhattan,Inwood,40.867684,-73.921210,3,Supermarket,Pharmacy,Grocery Store,Organic Grocery,Convenience Store
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0,Supermarket,Pharmacy,Organic Grocery,Grocery Store,Convenience Store


Examining clusters

In [138]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Chinatown,Supermarket,Organic Grocery,Pharmacy,Grocery Store,Convenience Store
4,Hamilton Heights,Supermarket,Pharmacy,Organic Grocery,Grocery Store,Convenience Store
6,Central Harlem,Supermarket,Pharmacy,Organic Grocery,Grocery Store,Convenience Store
9,Yorkville,Pharmacy,Supermarket,Organic Grocery,Grocery Store,Convenience Store
10,Lenox Hill,Grocery Store,Supermarket,Pharmacy,Organic Grocery,Convenience Store
12,Upper West Side,Supermarket,Pharmacy,Organic Grocery,Grocery Store,Convenience Store
14,Clinton,Supermarket,Organic Grocery,Grocery Store,Pharmacy,Convenience Store
15,Midtown,Grocery Store,Supermarket,Pharmacy,Organic Grocery,Convenience Store
17,Chelsea,Grocery Store,Supermarket,Pharmacy,Organic Grocery,Convenience Store
18,Greenwich Village,Supermarket,Pharmacy,Organic Grocery,Grocery Store,Convenience Store


In [139]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
7,East Harlem,Pharmacy,Grocery Store,Convenience Store,Supermarket,Organic Grocery
8,Upper East Side,Grocery Store,Supermarket,Pharmacy,Organic Grocery,Convenience Store
13,Lincoln Square,Grocery Store,Pharmacy,Supermarket,Organic Grocery,Convenience Store
16,Murray Hill,Grocery Store,Pharmacy,Supermarket,Organic Grocery,Convenience Store
25,Manhattan Valley,Grocery Store,Supermarket,Pharmacy,Organic Grocery,Convenience Store
27,Gramercy,Grocery Store,Supermarket,Pharmacy,Organic Grocery,Convenience Store
28,Battery Park City,Grocery Store,Supermarket,Pharmacy,Organic Grocery,Convenience Store
31,Noho,Grocery Store,Supermarket,Pharmacy,Organic Grocery,Convenience Store
34,Sutton Place,Grocery Store,Supermarket,Pharmacy,Organic Grocery,Convenience Store


In [140]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Marble Hill,Pharmacy,Supermarket,Organic Grocery,Grocery Store,Convenience Store


In [141]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,Washington Heights,Grocery Store,Supermarket,Pharmacy,Organic Grocery,Convenience Store
3,Inwood,Supermarket,Pharmacy,Grocery Store,Organic Grocery,Convenience Store
5,Manhattanville,Supermarket,Grocery Store,Pharmacy,Organic Grocery,Convenience Store
26,Morningside Heights,Supermarket,Pharmacy,Grocery Store,Organic Grocery,Convenience Store
30,Carnegie Hill,Grocery Store,Supermarket,Pharmacy,Organic Grocery,Convenience Store


In [142]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
11,Roosevelt Island,Supermarket,Pharmacy,Organic Grocery,Grocery Store,Convenience Store


### Conclusion
**Currently, the COVID-19 virus has spread across the globe, resulting in the latest pandemic. Since March 2020, New York City has seen rapid increase in confirmed cases, leading to a "shelter in place" order issued by the state officials. This means that healthy or sick, all New Yorkers must stay home unless they are essential workers or need urgent health care. All non-essential businesses that are normally open to the public must remain closed. Which neighborhoods are more affected by the latest orders? It would be helpful to understand which businesses are closed / remain open, in order to better redirect resources to support each of the neighborhoods.**